In [1]:
# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# Ollama 서버 실행 (백그라운드 유지)
import subprocess
ollama_server = subprocess.Popen(["ollama", "serve"])

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
# DeepSeek 1.3B 모델 설치
!ollama pull deepseek-coder:1.3b

In [2]:
# Gradio 인터페이스 및 requests 라이브러리 불러오기
import gradio as gr
import requests

# 사용자가 입력한 메시지를 모델에게 전달하고 응답을 받아오는 함수 정의
def chat_with_model(message, history):
    try:
        # 로컬에서 실행 중인 Ollama API로 POST 요청 보내기
        response = requests.post(
            "http://localhost:11434/api/generate",  # Ollama API 엔드포인트
            json={
                "model": "deepseek-coder:1.3b",     # 사용할 모델 이름
                "prompt": message,                  # 사용자 입력 프롬프트
                "stream": False                     # 스트리밍 응답 사용 안 함
            }
        )
        # 응답에서 'response' 필드 값을 추출하고, 앞뒤 공백 제거
        output = response.json().get("response", "").strip()
        if not output:
            output = "모델 응답이 없습니다."
    except Exception as e:
        # 예외 발생 시 오류 메시지를 출력
        output = f"오류 발생: {str(e)}"

    # 채팅 기록에 사용자 입력과 모델 응답을 추가
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": output})
    return "", history  # 입력창은 비우고, 채팅 기록은 반환

# Gradio Blocks 인터페이스 생성
with gr.Blocks(theme=gr.themes.Base()) as demo:
    gr.Markdown("## 🧠 DeepSeek Chatbot")  # 타이틀 출력
    chatbot = gr.Chatbot(                  # 채팅 UI 컴포넌트
        type='messages',
        bubble_full_width=False,           # 말풍선 너비 자동 조절
        show_copy_button=True              # 복사 버튼 표시
    )
    msg = gr.Textbox(                      # 사용자 입력 텍스트박스
        placeholder="메시지를 입력하세요...",
        show_label=False
    )
    # Enter 키 입력 시 chat_with_model 함수 호출
    msg.submit(chat_with_model, [msg, chatbot], [msg, chatbot])

# 기존 포트(7861)를 점유하고 있는 프로세스를 강제로 종료 (서버 재시작 방지 목적)
!fuser -k 7861/tcp

# Gradio 앱 실행 (로컬 및 외부에서 접근 가능하게 함)
demo.launch(server_name="0.0.0.0", server_port=7861)


/tmp/ipython-input-1309252666.py:33: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(                  # 채팅 UI 컴포넌트


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://81c10c217cf852a053.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
# Cloudflared 설치
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

# 터널 실행 (Gradio 포트 7861에 연결)
!./cloudflared tunnel --url http://localhost:7861 > tunnel_log.txt 2>&1 & sleep 5

# URL 추출
import re
with open("tunnel_log.txt", "r") as f:
    log = f.read()

url_match = re.search(r"https://[a-z0-9\-]+\.trycloudflare\.com", log)
if url_match:
    print("퍼블릭 URL:", url_match.group(0))
else:
    print("URL 생성 실패")


cloudflared: Text file busy
퍼블릭 URL: https://bluetooth-bay-om-ea.trycloudflare.com
